In [1]:
import os.path
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import math
import torch
import pickle
import sys
from deap import base, creator, tools, algorithms
from torch.nn.functional import normalize as normalize
from es.utility import eaGenerateUpdateBatched
from es.LMMAEGES import LMMAEGES
from es.evaluator import Evaluator
import torch
from torchvision import datasets, transforms
from utility.HebbianNetworkClassifier import HebbianNetworkClassifier
import matplotlib.pyplot as plt
import numpy as np
import pickle
import wandb
from es.utility import create_torch_stats, TorchHallOfFame, plot_res


# set seed for reproducibility
torch.manual_seed(42)
torch.mps.manual_seed(42)

# setting the correct device
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
print('Using device:', device)

Using device: mps


In [2]:

# Define the transformation to apply to the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

batch_size = 64
# Download and load the MNIST training dataset
trn_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
trn_loader = torch.utils.data.DataLoader(trn_dataset, batch_size=batch_size, shuffle=True)

# Split the training dataset into training and validation datasets
train_size = int(0.85 * len(trn_dataset))
val_size = len(trn_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(trn_dataset, [train_size, val_size])
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1024, shuffle=True)


# Download and load the MNIST test dataset
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1024, shuffle=False)

model_size = [784, 512, 256, 10]
model = HebbianNetworkClassifier(
    model_size, 
    device=device, 
    init="linear",
    dropout=0.1,
    bias=False,
    activation=torch.functional.F.relu,
    rank=1,
    neuron_centric=True
)

model.eval()

loss_fn = torch.nn.CrossEntropyLoss()
evaluator = Evaluator(loss_fn, device, 10, model)

In [3]:
num_param = sum(p.numel() for p in model.parameters())
print(f"Number of parameters: {num_param}")

flatten = model.get_flatten_params()
model.set_params_flatten(flatten)

Number of parameters: 7016


In [4]:
# create fitness and individual from deap
creator.create("Fitness", base.Fitness, weights=(-1.0,))
creator.create("Individual", torch.Tensor, fitness=creator.Fitness)

In [13]:

strategy = LMMAEGES(
    n=num_param,
    evaluator=evaluator,
    sigma=1,
    device=device,
    use_levy_flight=True,
    use_real_gradient=False,
    beta1=0.9
)

# creating toolbox
toolbox = base.Toolbox()
toolbox.register("generate", strategy.generate, creator.Individual)
toolbox.register("update", strategy.update)
toolbox.register("evaluate", evaluator.evaluate)
toolbox.register("set_batch", evaluator.set_batch)
toolbox.register("set_best_weights", evaluator.set_best_weights)

# creating stats and hall of fame
stats = create_torch_stats()
hof = TorchHallOfFame(creator.Individual, 1)

# running the algorithm
final_pop, logbook = eaGenerateUpdateBatched(toolbox, train_loader=trn_loader, halloffame=hof, ngen=1, stats=stats, verbose=True)
    
# plotting the results if in notebook
if 'ipykernel' in sys.modules:
    plot_res(logbook)

gen	nevals	avg               	std              	min              	max              
0  	30    	19.565372467041016	7.221658706665039	7.381247520446777	42.54444885253906
1  	30    	18.116270065307617	7.247162818908691	6.79877233505249 	41.87040710449219
2  	30    	15.589255332946777	8.915639877319336	7.138774871826172	45.96677780151367
3  	30    	17.88716697692871 	5.599076747894287	9.578529357910156	32.79813003540039
4  	30    	13.939099311828613	6.608221054077148	7.428796768188477	36.07257843017578
5  	30    	11.892212867736816	4.252121448516846	6.319305419921875	25.656055450439453
6  	30    	12.242356300354004	3.7208125591278076	7.6853718757629395	23.254261016845703
7  	30    	12.62055778503418 	3.735905647277832 	6.501086235046387 	20.589082717895508
8  	30    	10.367936134338379	2.8859641551971436	5.920206069946289 	19.219480514526367
9  	30    	12.130637168884277	3.216914176940918 	7.942584991455078 	23.469751358032227
10 	30    	7.824765205383301 	2.062609910964966 	4.368635654449

In [10]:
best = hof.items[0]
model.set_params_flatten(best)

In [11]:
train_loss, val_loss, test_loss, train_accuracy, val_accuracy, test_accuracy, confusion_matrix = model.hebbian_train_loop(
    loss_fn, train_loader, val_loader, test_loader, max_iter=100, log=False,
) # used trn_loader instead of train_loader and None instead of val_loader

Test: 100%|██████████| 10/10 [00:03<00:00,  3.33batch/s, Loss=1.06]                                                                                      


In [12]:
print(f'Test accuracy: {test_accuracy}')

Test accuracy: 0.6026586415816326
